In [ ]:
import torch.nn as nn
import torch.optim as optim
from datetime import datetime

from feed_forward_network import FeedForwardNetwork
from preprocessing import *
from utils import *
from features import *

if torch.backends.mps.is_available():
    device = torch.device("mps")
    torch.mps.set_per_process_memory_fraction(0.)
else:
    device = torch.device("cpu")
device

In [ ]:
raw_data = pd.read_csv('consumption_and_temperatures.csv')
raw_data['timestamp'] = pd.to_datetime(raw_data['timestamp'])

seq_len = 0
scale_output=True
target_column = 'NO1_consumption'

features_to_add = [
    (   
        pick_location_data,
        { 'loc': [1] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {
            "shift_max": 10,
            "column_to_shift": "NO1_temperature",
            "new_column_name": "temp"
        }
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO1_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    ),
    (
        get_yesterday_target_mean,
        {'target': 'NO1_consumption'}
    )
]

In [ ]:
(X_train, y_train), (X_val, y_val), (X_test, y_test), (scalerInputMethod, scalerOutputMethod), df_target = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add,
        seq_len=seq_len,
        scale_output=scale_output
    )

seq_len = len(raw_data) - len(X_train) - len(X_test) - len(X_val)
model = FeedForwardNetwork(input_size=X_train.shape[1])
model.to(device)

lr = 0.01
num_epochs=100

loss_func = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=lr)

model, losses, val_loss, opt = fit(
    model, 
    train_set=(X_train, y_train),
    val_set=(X_val, y_val),
    opt=opt,
    loss_func=loss_func,
    num_epochs=num_epochs,
    device=device
)

path = f'models/FeedForwardNet-{str(datetime.now().date())}-loss-{str(losses[len(losses)-1])}'
torch.save(model.state_dict(), path)

plt.plot(losses, label="train loss")
plt.plot(val_loss, label="val loss")
plt.legend()

y_test, y_pred = predict(model, scalerOutputMethod, (X_test, y_test))

plot_error_by_hour_for_test_set(y_test, y_pred, start_hour=df_target['timestamp'].dt.hour.iloc[seq_len])

for k in range(5):
    make_forecast(y_pred=y_pred, df_target=df_target, seq_len=seq_len+1)

### Model of zone 1 applied on zone 2

In [ ]:
features_to_add2 = [
    (   
        pick_location_data,
        { 'loc': [2] }
    ),
    (
        add_season_columns, 
        {}
    ),
    (
        shift_data, 
        {
            "shift_max": 10,
            "column_to_shift": "NO2_temperature",
            "new_column_name": "temp"
        }
    ),
    (
        shift_data, 
        {   
            "shift_min": 24,
            "shift_max": 24,
            "column_to_shift": "NO2_consumption",
            "new_column_name": "consum"
        },
    ),
    (
        add_hour_columns,
        {}
    ),
    (
        get_yesterday_target_mean,
        {'target': 'NO2_consumption'}
    )
]

In [ ]:
(X_train2, y_train2), (X_val2, y_val2), (X_test2, y_test2), (scalerInputMethod2, scalerOutputMethod2), df_target2 = general_preprocessing(
        raw_data, 
        features_to_add=features_to_add2,
        seq_len=seq_len,
        scale_output=scale_output,
        target_column='NO2_consumption',
        features_to_scale=['NO2_temperature'],
    )

_, y_pred12 = predict(model, scalerOutputMethod2, (X_test2, y_test2))
model2 = FeedForwardNetwork(input_size=X_train.shape[1])
model2.to(device)

loss_func2 = nn.MSELoss()
opt2 = optim.Adam(model2.parameters(), lr=lr)
model2, losses2, val_loss2, opt2 = fit(
    model2, 
    train_set=(X_train2, y_train2),
    val_set=(X_val2, y_val2),
    opt=opt2,
    loss_func=loss_func2,
    num_epochs=num_epochs,
    device=device
)

y_test2, y_pred22 = predict(model2, scalerOutputMethod2, (X_test2, y_test2))
preds = {
    'ffn_zone1': torch.Tensor(y_pred12),
    'ffn_zone2': torch.Tensor(y_pred22),
}

targets = {
    'ffn_zone1': torch.Tensor(y_test2),
    'ffn_zone2': torch.Tensor(y_test2),
}
summary_bar_plot(preds=preds, targets=targets)